<a href="https://colab.research.google.com/github/JContro/pytorch_projects/blob/master/Flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import numpy as np


In [0]:
data_dir = '/content/drive/My Drive/Data/flowers'

In [0]:
import h5py 
import os, sys

In [0]:
class dataset_h5(torch.utils.data.Dataset):
    def __init__(self, in_file, transform=None):
        super(dataset_h5, self).__init__()
 
        self.file = h5py.File(in_file, 'r')
        self.transform = transform
 
    def __getitem__(self, index):
        x = torch.from_numpy(self.file['X_train'][index, :].transpose(2,0,1)).float()
        y = np.argmax(self.file['y_train'][index, :])
        
        # Preprocessing each image
        if self.transform is not None:
            x = self.transform(x)        
        
        return (x, y)
 
    def __len__(self):
        return self.file['X_train'].shape[0]

In [0]:
transform = None

In [0]:
dataset = dataset_h5("/content/drive/My Drive/Data/flowers.h5",transform=transform)
dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=32, shuffle=True,
        drop_last=True)

In [0]:
# Run this to test your data loader
images, labels = next(iter(dataloader))
labels

tensor([0, 1, 4, 4, 4, 1, 0, 4, 4, 1, 3, 0, 3, 1, 1, 0, 3, 0, 3, 1, 4, 0, 1, 1,
        1, 1, 4, 1, 2, 3, 1, 4])

In [0]:
class Net(nn.Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = nn.Sequential(
            # Defining a 2D convolution layer
            nn.Conv2d(3, 6, kernel_size=5, stride=1, padding=1),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(6),
            nn.ReLU(inplace=True),
            
            # Defining another 2D convolution layer
            nn.Conv2d(6, 12, kernel_size=5, stride=1, padding=1),
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(12),
            nn.ReLU(inplace=True),
            

            nn.Conv2d(12, 24, kernel_size=7, stride=1, padding=1),
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(24),
            nn.ReLU(inplace=True),
            

            nn.Conv2d(24, 48, kernel_size=5, stride=1, padding=1),
            nn.MaxPool2d(2,2),
            nn.BatchNorm2d(48),
            nn.ReLU(inplace=True),
            
        )

        self.linear_layers = nn.Sequential(
            nn.Linear(2881200, 4*63),
            nn.Linear(4*63, 63),
            nn.Linear(63, 5)

        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        print(x.shape)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [0]:
# defining the model
model = Net()
# defining the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = nn.CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

RuntimeError: ignored

In [0]:
# del model
# torch.cuda.empty_cache()

In [0]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])

In [0]:
params

726124343

In [0]:
def train(epoch):
    model.train()
    tr_loss = 0
    train_losses = []
    # getting the training set
    c = 0

    for images, labels in dataloader:
      c += 1
    
      # converting the data into GPU format
      if torch.cuda.is_available():
          images = images.cuda()
          labels = labels.cuda()
          

      # clearing the Gradients of the model parameters
      optimizer.zero_grad()
    
      # prediction for training and validation set
      output = model(images)
  
      # computing the training and validation loss
      loss_train = criterion(output, labels)
      train_losses.append(loss_train)
      

      # computing the updated weights of all the model parameters
      loss_train.backward()
      optimizer.step()
      tr_loss = loss_train.item()
      if c%10 == 0:
        print(f"The epoch {epoch + 1} has a loss of {tr_loss}")

In [0]:
epochs = 20
for e in range(epochs):
  train(e)

torch.Size([32, 48, 245, 245])


RuntimeError: ignored

In [0]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=10d35b10d0e4b24ef1c28e4629827c8b1b0b2064912ca4584bf6db8dedd187e6
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 25.2 GB  | Proc size: 4.7 GB
GPU RAM Free: 16MB | Used: 11425MB | Util 100% | Total 11441MB


In [0]:
!kill -9 -1

In [0]:
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

NameError: ignored